In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import codecs, json
from tqdm import tqdm
import numpy as np
from datetime import datetime
import time
import math

import sys
sys.path.append('drive/MyDrive/BmiResearch')
from pipeline_structure.signal_processing.asr_utils import clean_windows, asr_calibrate, asr_process, apply_asr

from constants import constants

In [ ]:
output_path = f'{constants.PREPROCESSED_DATASET_PATH}/ASR'
output_path

'drive/MyDrive/BmiResearch/data/datasets/preprocessed/a_walk_in_the_park/ASR'

In [ ]:
for subject in sorted(os.listdir(constants.BASE_DATASET_PATH)):
  print(subject)

  experiment_settings = dict()

  experiment_settings['general_params'] = {'low_filter':constants.low_filter,
                                          'high_filter':constants.high_filter,
                                          'frequency':constants.freq,
                                          'minutes_for_test':constants.minutes_for_test,
                                          'window_size':constants.window_size,
                                          'overlap':constants.overlap,
                                          'EEG_CHANNELS':constants.EEG_CHANNELS}
  experiment_settings['signal_processing'] = 'ASR'
  experiment_settings['DateTime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  experiment_settings['BASE_DATASET_PATH'] = constants.BASE_DATASET_PATH
  experiment_settings['OUTPUT_PATH'] = output_path

  output_path_subject = (f'{output_path}/{subject}')
  os.makedirs(output_path_subject)

  experiment_settings[subject] = dict()

  # Fit ASR
  fit_df = codecs.open(f'{constants.BASE_DATASET_PATH}/{subject}/X_fit.json', 'r', encoding='utf-8').read()
  fit_df = json.loads(fit_df)
  fit_df = np.array(fit_df)
  pre_cleaned, _ = clean_windows(fit_df.T, constants.freq, max_bad_chans=0.1)
  M, T = asr_calibrate(pre_cleaned, constants.freq, cutoff=20)
  fit_df_s_asr = asr_process(fit_df.T, constants.freq, M, T)
  fit_df_s_asr = fit_df_s_asr.T
  fit_df_s_asr = fit_df_s_asr.tolist()
  json.dump(fit_df_s_asr, codecs.open(f'{output_path_subject}/X_fit.json', 'w', encoding='utf-8'),
    separators=(',', ':'),
    sort_keys=True,
    indent=4)

  # process test
  chanks_test = codecs.open(f'{constants.BASE_DATASET_PATH}/{subject}/X_test_chunks.json', 'r', encoding='utf-8').read()
  chanks_test = json.loads(chanks_test)
  chanks_test = np.array(chanks_test)

  starttime = time.perf_counter()
  chanks_test_asr = apply_asr(chanks_test, constants.freq, M, T)
  proc_end = time.perf_counter() - starttime
  proc_1ch_s = round(proc_end / chanks_test_asr.shape[0], 5)
  experiment_settings[subject]['proc_1_test_ch_s'] = proc_1ch_s
  experiment_settings[subject]['proc_test_s'] = round(proc_end, 5)
  experiment_settings[subject]['len_test'] = chanks_test.shape[0]
  experiment_settings[subject]['output_path_subject'] = output_path_subject

  chanks_test_asr = chanks_test_asr.tolist()
  json.dump(chanks_test_asr, codecs.open(f'{output_path_subject}/X_test_chunks.json', 'w', encoding='utf-8'),
    separators=(',', ':'),
    sort_keys=True,
    indent=4)

  # process train
  chanks_train = codecs.open(f'{constants.BASE_DATASET_PATH}/{subject}/X_train_chunks.json', 'r', encoding='utf-8').read()
  chanks_train = json.loads(chanks_train)
  chanks_train = np.array(chanks_train)

  chanks_train_asr = apply_asr(chanks_train, constants.freq, M, T)
  chanks_train_asr = chanks_train_asr.tolist()
  json.dump(chanks_train_asr, codecs.open(f'{output_path_subject}/X_train_chunks.json', 'w', encoding='utf-8'),
    separators=(',', ':'),
    sort_keys=True,
    indent=4)

  json.dump(experiment_settings, codecs.open(f'{output_path_subject}/experiment_settings.json', 'w', encoding='utf-8'),
    separators=(',', ':'),
    sort_keys=True,
    indent=4)

sub-016
[ASR] Determining channel-wise rejection thresholds
[ASR] Calibrating...
[ASR] Calibration done.
[apply_asr]
data_chanks_list_train shape =  (2996, 33, 100)


100%|██████████| 2996/2996 [00:53<00:00, 56.06it/s]


[apply_asr]
data_chanks_list_train shape =  (24175, 33, 100)


100%|██████████| 24175/24175 [06:56<00:00, 58.02it/s]


sub-017
[ASR] Determining channel-wise rejection thresholds
[ASR] Calibrating...
[ASR] Calibration done.
[apply_asr]
data_chanks_list_train shape =  (2996, 33, 100)


100%|██████████| 2996/2996 [00:53<00:00, 56.52it/s]


[apply_asr]
data_chanks_list_train shape =  (16436, 33, 100)


100%|██████████| 16436/16436 [04:34<00:00, 59.97it/s]
